In [1]:
# kaggle dependencies
########################
# pip install fastkaggle
# pip install wandb
# pip install polars
# pip install datasets
# pip install scikit-learn
# pip install evaluate

In [2]:
# | default_exp 00_base_run

In [3]:
#| export 
import os

iskaggle = os.path.exists("/kaggle/input")
isremote = os.path.exists("/home/ubuntu")

In [4]:
#| export 
import wandb
import argparse
import os
import shutil
import fastkaggle
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from datasets import Dataset
import torch  # base
import torch.nn.functional as F
import json
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from pydantic import BaseModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset as TorchDataset
import evaluate
from torch.utils.data import DataLoader
from datetime import datetime
import subprocess
from sklearn.metrics import accuracy_score
from typing import List, Optional, Dict, Tuple, Union, Literal

## LLM Response Scoring with BERT

This notebook is for the [llm-classification-finetuning](https://www.kaggle.com/competitions/llm-classification-finetuning) competition on kaggle. It's a quick fine-tune of the `bert-base-uncased` model to predict which LLM response is preferrable. There are probably better models and approaches for this, but BERT does pretty well on its own without a whole lot of intervetion.

For me, this was more of a quick experiment in getting some external dependincies set up in a kaggle `code competition` notebook.

- To get additional libraries installed, open the notebook in kaggle and select `Install Dependencies` from the `Add-On` menu.

- To run the BERT model offline, add this dataset to your notebook dependencies:
  - https://www.kaggle.com/datasets/xhlulu/huggingface-bert

You can see later on how to reference the local model location in the kaggle notebook.


Handle the GPU handoff for all the different machines


In [5]:
#| export 

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")
    print("Using CPU")

print(f"Device: {device}")

Using MPS (Apple Silicon GPU)
Device: mps


In [6]:
#| export 
if not iskaggle and not isremote:
    data_base_path = Path("./data")
    comp_name = "llm-classification-finetuning"
    datapath = data_base_path / comp_name
    if not os.path.exists(datapath) and not datapath.exists():
        install_path = fastkaggle.setup_comp(comp_name)
        shutil.move(install_path, datapath)

In [7]:
#| export 
if isremote:
    if os.getcwd() == "/home/ubuntu":
        os.chdir("./llm-classification-finetuning")

### Set up Kaggle/Local Env


In [8]:
#| export 
WANDB_PROJECT_NAME = "kaggle-llm-classification"


def setup_environment():
    """Detect environment and set up paths for both local and Kaggle"""

    if iskaggle:
        print("Running on Kaggle")

        INPUT_DIR = "/kaggle/input/llm-classification-finetuning"
        OUTPUT_DIR = "/kaggle/working"
        MODEL_DIR = "/kaggle/working/models"

        os.environ["WANDB_MODE"] = "disabled"
    else:
        print("💻 Running locally")

        INPUT_DIR = "./data/llm-classification-finetuning"
        OUTPUT_DIR = "./output"
        MODEL_DIR = "./models"

        os.environ["WANDB_PROJECT"] = WANDB_PROJECT_NAME
        os.environ["WANDB_LOG_MODEL"] = "false"
        os.environ["WANDB_WATCH"] = "false"
        # os.environ["WANDB_MODE"] = "disabled"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(MODEL_DIR, exist_ok=True)
    # to kill warning when running in notebooks
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    return INPUT_DIR, OUTPUT_DIR, MODEL_DIR


INPUT_DIR, OUTPUT_DIR, MODEL_DIR = setup_environment()
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Model directory: {MODEL_DIR}")

💻 Running locally
Input directory: ./data/llm-classification-finetuning
Output directory: ./output
Model directory: ./models


### Load Data


In [9]:
#| export 
try:
    df_train = pl.read_csv(f"{INPUT_DIR}/train.csv")
    df_test = pl.read_csv(f"{INPUT_DIR}/test.csv")
    df_sample = pl.read_csv(f"{INPUT_DIR}/sample_submission.csv")

    print("Data loaded successfully!")
    print(f"Train: {df_train.shape}")
    print(f"Test: {df_test.shape}")
    print(f"Sample submission: {df_sample.shape}")

except FileNotFoundError as e:
    print(f" Data file not found: {e}")
    print(f" Make sure data is in: {INPUT_DIR}")

    if os.path.exists(INPUT_DIR):
        files = os.listdir(INPUT_DIR)
        print(f"📁 Files in {INPUT_DIR}: {files}")

Data loaded successfully!
Train: (57477, 9)
Test: (3, 4)
Sample submission: (3, 4)


In [10]:

pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [11]:
df_train.sample(n=10)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
568663419,"""claude-instant-1""","""gpt-4-0314""","""[""are there places in the US where you can rent a dog for a day?""]""","""[""Yes, there are some places in the US where you can rent a dog for a day or for a few hours. Here are some examples:\n\n\u2022 Bark and Meow - This service operates in several cities like New York, Los Angeles, San Francisco, etc. They offer day rentals and overnight rentals of various dog breeds.\n\n\u2022 Waghotels - They have locations across the US and offer day rentals of dogs. You can choose from breeds like Labs, Goldens, French Bulldogs, etc. \n\n\u2022 Sniffspot - This is an online platform where dog owners can list their dogs for rent. You can search for day rentals of dogs in your area and book directly with the owners.\n\n\u2022 Local dog sitting and walking companies - Some local dog walking and pet sitting businesses also offer day rentals of dogs as an added service. You can search online for companies in your city that may offer this.\n\n\u2022 Animal shelters - Some animal shelters and rescue organizations will let you borrow a dog for the day as a means of promoting …","""[""Yes, there are places in the US where you can rent a dog for a day, often through programs called \""Dog for a Day\"" or \""Rent-a-Dog.\"" These programs are typically run by animal shelters and rescue organizations, allowing people to take a dog out for a day or a few hours to provide them with exercise, socialization, and a break from the shelter environment. Some popular locations offering such programs include:\n\n1. Austin Animal Center in Austin, Texas\n2. Maui Humane Society in Puunene, Hawaii\n3. Nashville Humane Association in Nashville, Tennessee\n4. Richmond Animal Care and Control in Richmond, Virginia\n5. Humane Society of Broward County in Fort Lauderdale, Florida\n\nBefore participating in these programs, it is essential to check the specific requirements and guidelines of your local shelter or rescue organization. While the primary purpose of these programs is to benefit the dogs, it can also be a great way for potential adopters to spend time with a dog and see if they a…",0,1,0
1194999683,"""stripedhyena-nous-7b""","""vicuna-33b""","""[""Write 8500 [extremely detailed, overly dialogue-laden words]\n\nIt was ironic the way he focused on small details when a huge deal was going down. Or maybe it wasn't. He had never been great with the English grammar thing, he wasn't sure what the actual definition of irony was. Still, it seemed odd to him that out of all memories he could have savoured, it was the music. Not the smoke. Not the drugs. Not the making love that just finished. The music. Not even music he liked, just Gilesy music.\n\nHe was in Giles' apartment listening to music.\n\nHe was in Giles' bed in Giles' apartment listening to music.\n\nHe was naked in Giles' bed in Giles' apartment listening to music.\n\nXander Harris was drunk and high on 500 or so different types of weed Giles somehow knew how to get while he was naked in Giles' bed in Giles' apartment listening to music. Then again, so was Giles. To all of those things. Because they just finished doing eachother, causing their bodies to start to stick to eac…","""[""right. But you, Gilesy, you're one of the best things to happen to me. One of the best things. And the music. The music.\"" He gestured vaguely at the speakers, the room echoing with the sounds of Giles' eclectic taste. \""It's like... it's like, I feel like I'm finally home.\""\n\nGiles, his eyes half-closed but his attention still fully on Xander, smiled softly. He scratched at the back of his neck, a faint blush tinging his cheeks. \""I'm glad you feel that way, Xander. I... I don't know how to put it into words, but hearing you say that... it means a lot to me.\""\n\n\""What does it mean?\"" Xander asked, his voice a little more serious than befo

In [12]:
df_train["prompt"][0]

'["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'

Try concatenating full conversations and full answer sets


In [13]:
#| export 
cols = ["prompt", "response_a", "response_b"]
if isinstance(df_train["prompt"][0], str):
    first_prompt_cell = df_train["prompt"][0]
    if "[" in first_prompt_cell and "]" in first_prompt_cell:
        df_train = df_train.with_columns([pl.col(col).str.json_decode() for col in cols])
        # explode
        df_train = df_train.explode(cols)
        # join
        # df_train = df_train.with_columns([pl.col(col).list.join(" ") for col in cols])

In [14]:
df_train["prompt"][0]

'Is it morally right to try to have a certain percentage of females on managerial positions?'

Remove any empty conversational turns:


In [15]:
#| export 
def null_clean(df: pl.DataFrame):
    df = df.drop_nulls(subset=["prompt", "response_a", "response_b"])
    return df

In [16]:
#| export 
df_train = null_clean(df_train)
df_test = null_clean(df_test)

In [17]:
df_train

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""Is it morally right to try to have a certain percentage of females on managerial positions?""","""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination. Here are some arguments in favor of and against such policies: **Arguments in favor:** 1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination. 2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole. 3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers. 4. **Role Modeling:** Increased v…","""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decision-making. Critics may argue that positions should be based purely on merit, and that quotas could potentially lead to tokenism or unqualified candidates being promoted. Morality can be subjective and differs from person to person based on their personal beliefs, cultural background, and values. It's a topic with valid arguments on both sides, and it's up to each individual or organization to decide what they believe is right.""",1,0,0
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""OK, does pineapple belong on a pizza? Relax and give me fun answer.""","""Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest. Let's slice into the debate with a zest of fun: **Team Pineapple:** ""Absolutely, yes! Pineapple on pizza is like a beach party in your mouth. The sweet juiciness of pineapple chunks frolicking with savory ham or bacon creates a flavor wave that surfs across the cheesy ocean of deliciousness. It's the Mardi Gras of pizzas, where sweet meets savory in a jubilant jamboree!"" **Team No-Pineapple:** ""No way, not in a million pizzas! Pineapple is a fruit that should be sunbathing on a tropical fruit platter, not freeloading on a sacred slice of pizza. The thought of warm, melty cheese conspiring with pineapple's sugary mischief is enough to make Italian ancestors turn in their tomato patches. Keep the pineapple in the fruit salad where it…","""As an AI, I don't eat, so I don't have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of the fruit has no place on a pizza. It's a pizza puzzle that might never be solved. So, whether pineapple belongs on a pizza or not, it truly depends on your taste buds!""",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has ta

Format for `BERT` by concatenating the prompt and both responses with `[SEP]` tokens in between


In [18]:
("[CLS]" + df_train["prompt"] + "[SEP]" + df_train["response_a"] + "[SEP]" + df_train["response_b"])[0]

'[CLS]Is it morally right to try to have a certain percentage of females on managerial positions?[SEP]The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in management can

In [19]:
#| export 
def apply_bert_fmt(df: pl.DataFrame):
    df = df.with_columns(
        (
           "[CLS]" + pl.col("prompt") + "[SEP]" + pl.col("response_a") + "[SEP]" + pl.col("response_b")
        ).alias("text")
    )
    return df

Condense to a single target column and remap to 0,1,2 as possible dep vars


In [20]:
#| export 
df_train = df_train.with_columns(
    pl.when(pl.col("winner_model_a") == 1)
    .then(pl.lit(0))
    .when(pl.col("winner_model_b") == 1)
    .then(pl.lit(1))
    .when(pl.col("winner_tie") == 1)
    .then(pl.lit(2))
    .otherwise(pl.lit(None))
    .alias("label")
)

In [21]:
#| export 
df_train = apply_bert_fmt(df_train)
df_test = apply_bert_fmt(df_test)

Split the training data into training and validation sets


In [22]:
df_train.sample(n=10)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,text
i64,str,str,str,str,str,i64,i64,i64,i32,str
1516395716,"""llama-2-70b-chat""","""openchat-3.5""","""respire et écris la chose la plus créative possible.""","""Je respire profondément et laisse mon esprit vagabonder. Une idée me vient, une histoire fantastique qui se déroule dans un monde mystérieux. Je m'inspire de la culture celte et de la mythologie grecque pour créer un récit unique et captivant. Dans un lointain passé, il y avait un royaume caché, invisible aux yeux des mortels. Les habitants de ce royaume étaient des êtres aux pouvoirs magiques, des druides, des fées et des créatures surnaturelles. Ils vivaient en harmonie avec la nature, respectant les forces ancestrales de la terre et des éléments. Au centre de ce royaume, il y avait un arbre sacré, un chêne colossal qui atteignait les cieux. Les druides disaient que l'arbre était le lien entre les mondes, le point de convergence entre le ciel, la terre et les enfers. Les fées venaient s'y reposer aux branches, baignant dans la lumière de la lune et se nourrissant des fruits de l'arbre. Mais un jour, un terrible maléfice tomba sur le royaume. Un sorcier maléfique, jaloux de la beau…","""Dans un univers parallèle où les oiseaux peuvent parler et les arbres sont dotés de mémoires, une aventure folklorique se déroule. Des légendes se répandent autour d'un mystérieux personnage, le Maître des Sentiers Oubliés. Ce dernier a le don unique d'être capable de voyager dans le temps et d'explorer les mondes imaginaires. Un jour, il décide de partager ses aventures avec les habitants de la forêt, en leur racontant les histoires les plus incroyables. Chaque soir, autour d'une grande feuille d'érable, les habitants se rassemblent pour écouter les récits du Maître des Sentiers Oubliés. Un jour, alors que le Maître raconte l'histoire d'un monde où les animaux parlent d'une voix magique, un jeune oiseau nommé Zephyr se trouve fasciné par les aventures du Maître. Zephyr décide de suivre le Maître dans ses explorations, espérant découvrir de nouvelles terres et nouvelles histoires. Le Maître, impressionné par la curiosité de Zephyr, l'accepte comme compagnon d'aventure. Ensemble, ils…",0,1,0,1,"""[CLS]respire et écris la chose la plus créative possible.[SEP]Je respire profondément et laisse mon esprit vagabonder. Une idée me vient, une histoire fantastique qui se déroule dans un monde mystérieux. Je m'inspire de la culture celte et de la mythologie grecque pour créer un récit unique et captivant. Dans un lointain passé, il y avait un royaume caché, invisible aux yeux des mortels. Les habitants de ce royaume étaient des êtres aux pouvoirs magiques, des druides, des fées et des créatures surnaturelles. Ils vivaient en harmonie avec la nature, respectant les forces ancestrales de la terre et des éléments. Au centre de ce royaume, il y avait un arbre sacré, un chêne colossal qui atteignait les cieux. Les druides disaient que l'arbre était le lien entre les mondes, le point de convergence entre le ciel, la terre et les enfers. Les fées venaient s'y reposer aux branches, baignant dans la lumière de la lune et se nourrissant des fruits de l'arbre. Mais un jour, un terrible maléfice…"
3069375861,"""claude-2.1""","""gpt-4-0314""","""Ok great.""","""You're welcome! Scripting SAP GUI logons and interactions can be very useful for automating tasks and processes. Let me know if you have any other questions as you develop your VBScripts for SAP. Things like running transactions, getting text from screens, inputting data, etc can all be done as well through the SAPGUI object model. There's lots you can do to integrate and automate with VBScript code.""","""I'm glad you found the information helpful! If you have any more questions or need further assistance, feel free to ask.""",0,0,1,2,"""[CLS]Ok great.[SEP]You're welcome! Scripting SAP GUI logons and interactions can be very useful for automating tasks a

## TODO:

- maybe use config files for some more model specific params


In [41]:

#| export 

def parse_args():
  parser = argparse.ArgumentParser(description="LLM Classification BERT Finetuning")
  parser.add_argument("--lr", type=float, default=2e-5, help="Learning rate for training")
  parser.add_argument("--bs", type=int, default=8, help="Batch size for training")
  parser.add_argument("--epoch", type=int, default=2, help="Number of training epochs")
  parser.add_argument("--max_steps", type=int, default=None, help="Maximum training steps")
  parser.add_argument("--model_path", type=str, default="answerdotai/ModernBERT-base", help="Model path or name")
  parser.add_argument("--output_dir", type=str, default=None, help="Output directory (overrides default)")
  parser.add_argument("--disable_wandb", action="store_true", help="Disable wandb logging")
  parser.add_argument("--max_len", type=int,default=8192, help="Max sequence length for model")

  args, unknown = parser.parse_known_args()
  return args

if __name__ == "__main__":
  args = parse_args()
else:
  class Args:
    lr = 2e-5
    bs = 8
    epochs = 2
    model_path = "answerdotai/ModernBERT-base"
    max_len = 8192

  args = Args()

In [24]:
#| export 
model_path = args.model_path
print("Loading BERT from Hugging Face...")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)
model.to(device)
model.device

Loading BERT from Hugging Face...


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='mps', index=0)

In [25]:
#| export 
class TokenizeDataset(TorchDataset):
    def __init__(self, texts, tokenizer):
      self.texts = texts
      self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       tokens = self.tokenizer.tokenize(self.texts[idx])
       return len(tokens)

In [26]:
#| export 
texts = df_train['text'].to_list()
dataset = TokenizeDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=1000, num_workers=4)
token_counts = []
for batch in dataloader:
    token_counts.extend(batch.tolist())

In [27]:
# | export
df_train = df_train.with_columns(pl.Series("token_count", token_counts))

In [28]:
# | export
# for now just filter anything over the context window
df_train.filter(pl.col("token_count") < 510)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,text,token_count
i64,str,str,str,str,str,i64,i64,i64,i32,str,i64
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits.""","""A marriage license and a marriage certificate are two different legal documents that have separate purposes. 1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the wedding ceremony takes place. The couple is required to meet certain criteria, such as being of a certain age or not being closely related. Once the license is issued, there's often a waiting period before the marriage ceremony can take place. The marriage license has to be signed by the couple, their witnesses, and the officiant conducting the marriage ceremony, then returned to the license issuer for recording. 2. Marriage Certificate: A marriage certificate, on the other hand, is a document that proves a marriage has legally taken place. It's issued after the marriage ceremony, once the signed marriage license has been returned and recorded. The marriage certificate includes details about the cou…",0,1,0,1,"""[CLS]What is the difference between marriage license and marriage certificate?[SEP]A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits.[SEP]A marriage license and a marriage certificate are two different legal documents that have separate purposes. 1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually…",388
53567,"""koala-13b""","""gpt-4-0613""","""What is the minimal time to get them? 1 day or 10 days?""","""The minimal time to get a marriage license and a marriage certificate in California can vary depending on the specific requirements and process in your county. Generally, you can obtain a marriage license and a marriage certificate in California within a few days of applying, but it is best to check with your county clerk's office for the specific requirements and process for your county. In some counties, you can obtain a marriage license and a marriage certificate on the same day, while in other counties, you may need to wait a few days. The process typically takes about 30 minutes to complete and you will need to bring identification, such as a driver's license or passport, and proof of your age, such as a birth certificate or a passport. It is important to note that some counties in California require a waiting period between the time you obtain the marriage license and the time of the mar

In [29]:
#| export 
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["text"], df_train["label"], test_size=0.1, random_state=42
)

In [30]:
train_texts.shape, val_texts.shape, train_labels.shape, val_labels.shape

((64205,), (7134,), (64205,), (7134,))

In [31]:
model

ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [32]:
tokenizer

PreTrainedTokenizerFast(name_or_path='answerdotai/ModernBERT-base', vocab_size=50280, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("|||IP_ADDRESS|||", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50257: AddedToken("                    

In [33]:
ex_enc = tokenizer(train_texts[0], truncation=True, padding=True)
ex_enc

{'input_ids': [50281, 50281, 2422, 500, 8879, 3525, 310, 432, 835, 32, 50282, 2422, 500, 8879, 3525, 310, 247, 11906, 4156, 3420, 2990, 8927, 432, 399, 41970, 13, 253, 5347, 2846, 273, 38372, 15, 733, 369, 10098, 327, 4596, 337, 13, 8441, 13, 407, 253, 38690, 273, 38372, 387, 253, 673, 13, 1500, 20323, 5516, 324, 10269, 35667, 38139, 1219, 596, 6451, 15, 380, 1416, 346, 2422, 500, 8879, 3525, 3, 30376, 281, 346, 510, 33489, 3, 275, 4383, 13, 14339, 281, 253, 49185, 33489, 15, 1219, 500, 8879, 3525, 310, 1929, 323, 697, 3907, 285, 2223, 4619, 7031, 273, 3668, 285, 1655, 13909, 13, 1097, 1561, 253, 10515, 5791, 285, 1475, 253, 1533, 15, 50282, 2422, 500, 8879, 3525, 310, 247, 3420, 2990, 1754, 275, 399, 41970, 13, 38372, 15, 733, 369, 10098, 275, 8441, 285, 556, 8228, 281, 320, 247, 2201, 4156, 3668, 6003, 13, 342, 2709, 5579, 8123, 285, 5865, 13498, 5277, 3668, 7031, 275, 26503, 13, 4383, 13, 285, 643, 11515, 15, 11474, 352, 310, 9633, 407, 253, 2208, 273, 38372, 13, 1219, 500, 8879, 35

### Load Dataset


In [34]:
#| export 
class LLMDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, labels=None, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    # encoding on the fly here due to issues with memory on kaggle
    # when pre-tokenizing
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {key: val.squeeze() for key, val in encoding.items()}

        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        return len(self.texts)

In [49]:
#| export 
train_dataset = LLMDataset(list(train_texts), tokenizer, list(train_labels), max_length=args.max_len)
val_dataset = LLMDataset(list(val_texts), tokenizer, list(val_labels), max_length=args.max_len)
test_dataset = LLMDataset(list(df_test["text"]), tokenizer, max_length=args.max_len )

In [50]:
#| export 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

### Train


In [51]:
#| export 
class TrainParams(BaseModel):
  learning_rate: float = 2e-5
  per_device_train_batch_size: int = 8
  per_device_eval_batch_size: int = 8
  num_train_epochs: Optional[int] = 2
  weight_decay: float = 0.01
  max_steps: Optional[int] = None
  eval_strategy: Literal["epoch", "steps", "no"] = "epoch"
  save_strategy: Literal["no", "steps", "epoch"] = "epoch"
  load_best_model_at_end: bool = True
  metric_for_best_model: Literal["accuracy"] = "accuracy"
  logging_steps: int = 10
  logging_first_step: bool = False
  dataloader_num_workers: int = 0 

In [52]:
# | export 
def get_train_params() -> TrainParams:
    """Get training parameters based on environment"""
    # remote gpu
    if isremote:
        return TrainParams(
            learning_rate=args.lr,
            per_device_train_batch_size=args.bs,
            per_device_eval_batch_size=args.bs,
            logging_steps=20,
        )
    # kaggle 
    if iskaggle:  
        return TrainParams(
            per_device_train_batch_size=args.bs,
            per_device_eval_batch_size=args.bs,
            load_best_model_at_end=True,
            save_strategy="no",
        )
    # local mps
    else:      
        return TrainParams(
            per_device_train_batch_size=args.bs,
            per_device_eval_batch_size=args.bs,
            max_steps=100,
            num_train_epochs=None,  
            eval_strategy="no",
            logging_steps=2,
            save_strategy="no",
                )

In [53]:
# if os.environ.get("WANDB_MODE") != "disabled":
#     import wandb
    
#     # Create simple tags
#     tags = [
#         f"lr_{training_args.learning_rate}",
#         f"bs_{training_args.per_device_train_batch_size}",
#         f"epochs_{training_args.num_train_epochs}" if hasattr(training_args, 'num_train_epochs') else f"steps_{training_args.max_steps}",
#         "test_run" if test_run else "full_run",
#         "kaggle" if iskaggle else ("remote" if 'isremote' in globals() and isremote else "local")
#     ]
    
#     wandb.run.tags = tags
#     print(f"🏷️  Tagged run with: {tags}")


In [54]:
# | export 
train_params = get_train_params()

In [55]:
#| export 
final_model_path = f"{MODEL_DIR}/final"

timestamp = datetime.now().strftime("%Y%m%d-%H%M")
run_name = f"bert-classification-{timestamp}"

In [56]:
# | export 
if os.environ.get("WANDB_MODE") != "disabled":
    import wandb
    
    tags = [
        f"model_{args.model_path.split('/')[-1]}",
        f"lr_{train_params.learning_rate}",
        f"bs_{train_params.per_device_train_batch_size}",
        f"epochs_{train_params.num_train_epochs}" if train_params.num_train_epochs else f"steps_{train_params.max_steps}",
    ]

    wandb.init(
        project=WANDB_PROJECT_NAME,
        name=run_name    ,
        tags=tags
    )
    print(f"🏷️  Tagged run with: {tags}")

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


🏷️  Tagged run with: ['model_ModernBERT-base', 'lr_2e-05', 'bs_8', 'steps_100']


In [57]:
# | export 
training_args = TrainingArguments(
    output_dir=f"{OUTPUT_DIR}/results",
    run_name=run_name,
    report_to="wandb" if os.environ.get("WANDB_MODE") != "disabled" else [],
    **train_params.model_dump(exclude_none=True)
)

In [58]:
# | export 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [59]:
# | export 
trainer.train()

KeyboardInterrupt: 

In [ ]:
# | export 
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
# if os.environ.get("WANDB_MODE") != "disabled":
#     wandb.log({"final_eval": eval_results})
#     wandb.save(f"{final_model_path}/*")
# wandb.finish()

### Inference


In [ ]:
# | export 
def load_model(model_path):
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer


model, tokenizer = load_model(final_model_path)
model.to(device)

text = "This is a test sentence"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs.to(device))
predictions = outputs.logits

In [ ]:
predictions

In [ ]:
# | export 
preds = F.softmax(predictions, dim=-1)

In [ ]:
preds

In [ ]:
# | export 
all_probabilities = []
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != "labels"}

        outputs = model(**inputs)
        probabilities = F.softmax(outputs.logits, dim=-1)
        all_probabilities.extend(probabilities.cpu().numpy())
final_probs = np.vstack(all_probabilities)

In [ ]:
final_probs

In [ ]:
final_probs[:, 0]

In [ ]:
# | export 
submission_df = df_test
submission_df = submission_df.with_columns(
    pl.lit(final_probs[:, 0]).alias("winner_model_a"),
    pl.lit(final_probs[:, 1]).alias("winner_model_b"),
    pl.lit(final_probs[:, 2]).alias("winner_tie"),
)
submission_df = submission_df[["id", "winner_model_a", "winner_model_b", "winner_tie"]]
submission_df

In [ ]:
# | export 
df_for_kaggle = submission_df.to_pandas()

In [ ]:
# | export 
df_for_kaggle.to_csv("submission.csv", index=False)
df_for_kaggle.head()

In [ ]:
from nbdev.export import nb_export

nb_export("20250709_unsplit_ds.ipynb", "llm_classifier")

---


### Push Notebook to Kaggle


In [ ]:
def push_notebook_cli():
    username = "peterbull"
    comp = "llm-classification-finetuning"
    notebook_file = "20250709_unsplit_ds.ipynb"
    metadata = {
        "id": f"{username}/llm-classification-bert-finetuning",
        "title": "LLM Classification BERT Finetuning",
        "code_file": notebook_file,
        "language": "python",
        "kernel_type": "notebook",
        "is_private": True,
        "enable_gpu": True,
        "enable_internet": False,  # required for kaggle code competition
        "dataset_sources": [],
        "competition_sources": [f"competitions/{comp}"],
        "kernel_sources": [],
    }

    with open("kernel-metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)

    if not os.path.exists(notebook_file):
        print(" Notebook file not found!")
        print(" Files in current directory:")
        for f in os.listdir("."):
            if f.endswith(".ipynb"):
                print(f"{f}")
        return

    print("Pushing to Kaggle...")
    try:
        result = subprocess.run(
            ["kaggle", "kernels", "push", "-p", "."], capture_output=True, text=True, timeout=300
        )

        if result.returncode == 0:
            print("✅ Notebook pushed successfully!")
            print(result.stdout)
            print(
                f"🔗 View at: https://www.kaggle.com/code/{username}/llm-classification-bert-finetuning"
            )
        else:
            print("Error pushing notebook:")
            print(result.stderr)

    except subprocess.TimeoutExpired:
        print("Upload timed out after 5 minutes")
    except FileNotFoundError:
        print("Kaggle CLI not found. Install with: pip install kaggle")
    except Exception as e:
        print(f"Unexpected error: {e}")


if not iskaggle:
    push_notebook_cli()